<h1 style="color:#00BFFF;">Exercise 1</h1>
<hr style="height:2px;border:none;color:#333;" />

<h2 style="color:#00BFFF;">Reliability Overtopping Exercise</h2>

# Index

---
[Introduction](#Introduction)    
[Loading data](#Loading-in-some-basic-packages-for-calculations-and-operations)   
[Parameters](#Parameters)    
[1.1 The Bretschneider equations](#The-Brestchneider-equations)

[1.2 The Iribarren parameter](#The-Iribarren-parameter)


#<h3 style="color:#00BFFF;">Introduction</h3>
<hr style="height:1.5px;border:none;color:#333;" />
*This is an optional exercise to check your understanding. Upon completion
of the exercise, please enter your answers into the attached Excel file to submit it for grading. A complete
solution will not be posted, instead you will use the Excel feedback sheet check your work.*


The goals of the first assignment are to:

* (re-)familiarise with Python exercises, 
* check your understanding of reliability analysis and waves/overtopping,
* introducing the realiability software (the Probabilistic Toolkit or OpenTURNS) 

Please read Chapters 4 and 5 of the lecture notes and the corresponding lecture before working on this notebook.

The overtopping manual can provide extra backround information on wave characterstics (chapter 1.4 - 1.4.3) and the overtopping equation (chapter 5.3.1); http://www.overtopping-manual.com/ 


<h3 style="color:#00BFFF;">Loading in some basic packages for calculations and operations</h3>
<hr style="height:1.5px;border:none;color:#333;" /> 

Click on the cell below and hit shift-enter to import the packages needed for this assignment.

In [ ]:
import numpy as np                   #importing numpy package for scientific computing
#import openturns as ot              #importing when using OpenTURNS 
#import openturns.viewer as viewer   #importing when using OpenTURNS
from matplotlib import pylab as plt  #importing matplotlib package for plots
#ot.Log.Show(ot.Log.NONE)            #to suppress all warnings when using OpenTURNS
import time                          #represents time-based objects
from tabulate import tabulate        #used to print tabular data in nicely formatted tables
import scipy.stats as scstats        #importing statistics functions
%matplotlib inline                   

#<h2 style="color:#00BFFF;">Parameters</h2>
<hr style="height:1.5px;border:none;color:#333;" />
All elevations are referenced to an arbitrary datum

* Coastal dike with simple cross section (no berm, foreshore irregularities, etc.);
* Design still water level: +7.0 m;
* Wind speed 30 m/s (measured at 10 meter elevation) with a fetch of 100 kilometers;
* Crest level: +14 m;
* Bottom level of the sea: –30.0 m;
* Outer slope is 1:5;
* Consider only 1 wind direction (perpendicular to the dike) and ignore all other effects like berms,
walls, wind-setup and overflow.



Below you can find all the given parameters that will be used in this assignment.

In [ ]:
design_water_level = 7
wind_speed = 30   
fetch = 100*1000   
crest_level = 14
bottom_level = -30
outer_slope = 1/5

The most important parameter for an overflow calculation is the freeboard of a dike, $R_{c}$ , defined as the difference between the crest elevation of a dike and the still water level. Thus, the freeboard is defined as positive
when the still water level is below the crest and negative when the still water level is above the crest. The water depth $d$ is defined as as the difference between the design water level and the bottom level. 

In [ ]:
freeboard = crest_level - design_water_level
water_depth = design_water_level - bottom_level

#print(freeboard)
#print(water_depth)

#<h2 style="color:#00BFFF;">1.1 The Bretschneider equations


</h2>
<hr style="height:1.5px;border:none;color:#333;" />

The significant wave height and significant wave period can be
estimated from wind speed using predictive equations calibrated with large data sets as done by Sverdrup, Munk and Bretschneider. In the Netherlands, these equations are referred to as the ‘Bretschneider equations’ and have been used in the design of flood defences for decades, although the parameters for these empirical equations have been repeatedly updated as new data becomes available. One version of these equations are given below as a function of wind speed, water depth and fetch:


$$
\tilde{H}=0.283 \tanh \left(0.530 \tilde{d}^{0.75}\right) \tanh \left(\frac{0.0125 \tilde{F}^{0.42}}{\tanh \left(0.530 \tilde{d}^{0.75}\right)}\right)\tag{1}
$$ 
$$
\tilde{T}=2.4 \pi \tanh \left(0.833 \tilde{d}^{0.375}\right) \tanh \left(\frac{0.077 \tilde{F}^{0.25}}{\tanh \left(0.833 \tilde{d}^{0.375}\right)}\right)\tag{2}
$$ 



Where $\tilde{H}$ , $\tilde{d}$, $\tilde{F}$ and $\tilde{T}$ are dimensionless coefficients:


$$
\tilde{H}=\frac{H_{s} g}{u_{10}^{2}}, \quad \tilde{d}=\frac{d g}{u_{10}^{2}},  \quad \tilde{F}=\frac{F g}{u_{10}^{2}}, \quad \tilde{T}=\frac{T_{s} g}{u_{10}} \tag{3}
$$


**<h3 style="color:#00BFFF;">Q1. Use the Bretschneider equations to calculate the significant wave height $H_{s}$ and significant wave
period $T_{s}$</h3>**
<hr style="height:1.5px;border:none;color:#333;" /> 

Define the Bretschneider equations.

In [ ]:
#1. Calculation of dimensionless water depth, dimensionless fetch, dimensionless wave height and dimensionless wave period

def bretschneider_dimensionless(water_depth,fetch,wind_speed):
    gravity = 9.81
    dimensionless_depth = water_depth*gravity/(wind_speed**2)
    dimensionless_fetch = fetch*gravity/(wind_speed**2)
    dimensionless_wave_height = (
        0.283*np.tanh(0.530*dimensionless_depth**0.75)
        *np.tanh(0.0125*dimensionless_fetch**0.42
                 /np.tanh(0.530*dimensionless_depth**0.75)
                 )
        )
    dimensionless_wave_period = (
        2.4*3.14159*np.tanh(0.833*dimensionless_depth**0.375)
        *np.tanh(0.077*dimensionless_fetch**0.25
                 /np.tanh(0.833*dimensionless_depth**0.375)
                 )
        )
    return dimensionless_wave_height,dimensionless_wave_period

print('dimensionless wave height=', bretschneider_dimensionless(water_depth,fetch,wind_speed)[0].round(2))
print('dimensionless wave period=', bretschneider_dimensionless(water_depth,fetch,wind_speed)[1].round(2))

#2. Calculation of significant wave height

def bretschneider_wave_height(water_depth,fetch,wind_speed):
    gravity = 9.81
    dimensionless_wave_height = (
        bretschneider_dimensionless(water_depth,fetch,wind_speed)[0]
        )
    significant_wave_height = (
        dimensionless_wave_height*(wind_speed**2)/gravity
        )
    return significant_wave_height

print('significant wave height=',  bretschneider_wave_height(water_depth,fetch,wind_speed).round(2), 'm')

#3. Calculation of significant wave period

def bretschneider_wave_period(water_depth,fetch,wind_speed):
    gravity = 9.81
    dimensionless_wave_period = (
        bretschneider_dimensionless(water_depth,fetch,wind_speed)[1]
        )
    significant_wave_period = (
        dimensionless_wave_period*wind_speed/gravity
        )
    return significant_wave_period

print('siginificant wave period=', bretschneider_wave_period(water_depth,fetch,wind_speed).round(2), 's')

dimensionless wave height= 0.05
dimensionless wave period= 2.73
significant wave height= 4.87 m
siginificant wave period= 8.36 s


In [ ]:
wave_height = bretschneider_wave_height(water_depth,fetch,wind_speed)
wave_period = bretschneider_wave_period(water_depth,fetch,wind_speed)
print('Q1: the significant wave height and period are {0:.2f} m '
      'and {1:.2f} s.'.format(wave_height,wave_period)
      )

Q1: the significant wave height and period are 4.87 m and 8.36 s.


#<h2 style="color:#00BFFF;">1.2 The Iribarren parameter


</h2>
<hr style="height:1.5px;border:none;color:#333;" />

Both run-up and overtopping depend strongly on whether or not the waves break onto the slope of the
structure. This can be described by the Iribarren parameter (also called surf similarity or breaker parameter).


$$
\xi_{m-1,0}=\tan \alpha / \sqrt{H_{m 0} / L_{m-1,0}} \tag{4}
$$ 


The Iribarren parameter relates wave steepness to the slope of a beach (or levee slope). In this case $H_{m 0}$ refers to the (spectral) significant wave height at the location of wave breaking. $H_{m 0}$ equals the significant wave height $H_{s}$, but is used when working with spectra. The significant wave height $H_{s}$ corresponds to the mean of the third of
the highest observed waves and is broadly used as the characteristic value for the wave height. The corresponding significant wave period $T_{s}$ is less frequently used. More common is the usage of $T_{m-1,0}$ or $T_{p}$ which again can be obtained from wave spectra analysis. As a rule of thumb, the following relationships can be used:

$$
T_{p}=1.08 * T_{s}=1.1 * T_{m,-1,0} \tag{5}
$$

Wave steepness is a useful way of evaluating wave characteristics and is defined as the ratio of a characteristic
wave height to a characteristic wave length, $s_{0p} = H_{m0}/L_{m-1,0}$. Swell seas will have a steepness of around 0.01,
whereas a typical wind sea 0.04 to 0.06 


The wave length $L_{m-1,0}$, equals:

$$L_{m-1,0}=
g T_{m-1,0}^{2} /(2 \pi) \tag{6}
$$

For small values of the Iribarren parameter (roughly $ξ$ < 2) the waves will form spilling or plunging
breakers on the slope. For larger values of $ξ$ the waves will not break, but rather surge onto the slope. The
resulting behaviour for both run-up and overtopping is different in these two cases. As a result, the prediction
formulas for run-up level and overtopping volumes usually have two different branches to describe these two
separate regimes.


**<h3 style="color:#00BFFF;">Q2. Calculate the wave steepness $s_{m-1,0}$ and Iribarren number $ξ_{m-1,0}$. What breaker type would you
expect?**
<hr style="height:1.5px;border:none;color:#333;" /> 

Define the functions.

In [ ]:
#Defining the functions for wave length, wave steepness and Iribarren number

def get_wave_length(wave_period):
    wave_length = 9.81*wave_period**2/2/3.14159
    return wave_length

def get_wave_steepness(wave_height,wave_period):
    wave_steepness = wave_height/get_wave_length(wave_period)
    return wave_steepness

def get_iribarren_number(outer_slope, wave_steepness):
    iribarren_number = outer_slope/np.sqrt(wave_steepness)
    return iribarren_number

In [ ]:
#1. Calculating the wave period
wave_period = 1.08*bretschneider_wave_period(water_depth,fetch,wind_speed)/1.10

#2. Calculating the wave steepness 
wave_steepness = get_wave_steepness(wave_height,wave_period)

#3. Calculating the Iribarren number
iribarren_number = get_iribarren_number(outer_slope,wave_steepness)

if iribarren_number >= 2:
  print('Waves are not breaking; breaker type= surging')
if iribarren_number < 2:
  print('Waves are breaking; breaker type= spilling or plunging')

print('Q2: the wave steepness is {0:.3f} and the Iribarren number is '
      '{1:.3f}, \n    using a spectral wave period of '
      '{2:.3f} s and {3:.0f} m wave length.'
      .format(wave_steepness,iribarren_number,wave_period,
                  get_wave_length(wave_period)
                  )
              )


Waves are breaking; breaker type= spilling or plunging
Q2: the wave steepness is 0.046 and the Iribarren number is 0.929, 
    using a spectral wave period of 8.205 s and 105 m wave length.


<h2 style="color:#00BFFF;">1.3 The overtopping discharge


</h2>
<hr style="height:1.5px;border:none;color:#333;" />

The theoretical shape of the overtopping formula can be derived
from an analysis of regular, breaking waves, and can then be extended to the use in irregular waves. Battjes (1974) found the resulting theoretical expression for the mean overtopping rate and expressed his results as a
rather complicated analytical function. However, Battjes’ results are only valid in the breaking wave regime, so for low Iribarren numbers. Like in the case of run-up, practical observations have shown that the overtopping reaches a maximum limit for non-breaking
waves (high Iribarren numbers).

For use in practice, van der Meer and Bruce (2014) provide a simple approximation of Battjes’ formula, and re-fitted the coefficients for use with $H_{m0}$ and $T{m−1,0}$. Their formula is as follows:


$$
\frac{q}{\sqrt{g \cdot H_{m 0}^{3}}}=\frac{0.023}{\sqrt{\tan \alpha}} \cdot \xi_{m-1,0} \cdot \exp \left\{-\left(2.7 \cdot \frac{R_{c}}{\xi_{m-1,0} \cdot H_{m 0}}\right)^{1.3}\right\} \tag{7}
$$

In the non-breaking wave regime there is no longer a theoretical basis for the shape of the formula. We have to rely on observations only, which have shown that for nonbreaking waves the mean overtopping rate reaches a maximum given by:


$$
\frac{q}{\sqrt{g \cdot H_{m 0}^{3}}}=0.09 \exp \left\{-\left(1.5 \cdot \frac{R_{c}}{H_{m 0}}\right)^{1.3}\right\} \tag{8}
$$


**<h3 style="color:#00BFFF;">Q3. Compute the overtopping discharge. Assume the dike is covered with grass.**
<hr style="height:1.5px;border:none;color:#333;" /> 

Define the overtopping equation. If the waves are breaking (Irribarren number > 2), there is a need to check the maximum. 

In [ ]:
#Defining the overtopping function for calculating the discharge q

def get_overtopping(wave_height,wave_period,outer_slope,freeboard,A,B):
    irribarren_number= get_iribarren_number(outer_slope, (get_wave_steepness(wave_height,wave_period)))
    discharge_cubic_meters = (
        np.sqrt(9.81*wave_height**3)*A/np.sqrt(outer_slope)
        *iribarren_number*np.exp(
            -(B*freeboard/iribarren_number/wave_height)**1.3
            )
        )
    discharge_liters = discharge_cubic_meters*1000
    print('overtopping discharge q=', discharge_liters.round(2))
    
    discharge_cubic_meters_maximum = (
        np.sqrt(9.81*wave_height**3)*0.09/np.sqrt(outer_slope)
        *iribarren_number*np.exp(
            -(1.5*freeboard/iribarren_number/wave_height)**1.3
            )
        )
    discharge_liters_maximum = discharge_cubic_meters_maximum*1000
    print('maximum overtopping discharge q=', discharge_liters_maximum.round(2))

#     if discharge_cubic_meters > discharge_liters_maximum:
#         print('Calculated q is greater than max (but the max value is not used).')

    if irribarren_number < 2:
      return discharge_liters
    if irribarren_number >= 2: 
      return discharge_liters_maximum


In [ ]:
print('Q3: the overtopping discharge is {0:.2f} L/m/s.'.format(
          get_overtopping(wave_height,wave_period,outer_slope,freeboard,0.023,2.7)
          )
      )

overtopping discharge q= 2.64
maximum overtopping discharge q= 317.95
Q3: the overtopping discharge is 2.64 L/m/s.


<h2 style="color:#00BFFF;">1.4 The overtopping discharge using design values


</h2>
<hr style="height:1.5px;border:none;color:#333;" />


According to van der Meer and Bruce (2014) the reliability of the coefficient values is as follows:

| General shape $$
\frac{q}{\sqrt{g\cdot H^{3}}}=A(\ldots) \exp -B(\ldots)^{1.3}
$$  | Equation 7 $$A$$ | Equation 7 $$B$$ | Equation 8 $$A$$ | Equation 8 $$B$$ |
|----------------------|---------|---------|---------|---------|
| Mean Value           | 0.023   | 2.7     | 0.09    | 1.5     |
| Standard Deviation   | 0.003   | 0.20    | 0.013   | 0.15    |
| Characteristic value | 0.026   | 2.5     | 0.1035  | 1.35    |



Now we will begin to incorporate uncertainty. The Table below shows two parameters that can be represented as random variables. The value to use for design depends on whether the variable is a resistance parameter or a load parameter. The design value of resistance is based on the characteristic value of the resistance, typically a 5% probability of exceedance. The design load is based on the characteristic value of the load, typically a 95% probability of exceedance.

| Variable         | Distribution | Mean  | Standard Deviation | 5% exc. value | 95% exc. value | Value to use for design |
|------------------|--------------|-------|--------------------|---------------|----------------|-------------------------|
| Crest elevation  | Normal       | 14 m  | 0.1 m              |               |                |                         |
| Bottom elevation | Normal       | -30 m | 3 m                |               |                |                         |

**<h3 style="color:#00BFFF;">Q4. Recompute the overtopping discharge with the appropriate design values.**
<hr style="height:1.5px;border:none;color:#333;" /> 

Be sure to use the proper values for the coefficients in the overtopping equation (See Table above; Table 5.2 in the lecture notes)

In [ ]:
crest_elevation_5= scstats.norm.ppf(0.05, 14, 0.1)  
#print('crest elevation: 5% exceedance value=', crest_elevation_5.round(2), 'm')
crest_elevation_95= scstats.norm.ppf(0.95, 14, 0.1)  
#print('crest elevation: 95% exceedance value=', crest_elevation_95.round(2), 'm')

bottom_elevation_5= scstats.norm.ppf(0.05, -30, 3)  
#print('bottom elevation: 5% exceedance value=', bottom_elevation_5.round(2), 'm')
bottom_elevation_95= scstats.norm.ppf(0.95, -30, 3)  
#print('bottom elevation: 95% exceedance value=', bottom_elevation_95.round(2), 'm')

#Determine the design values for the resistance and the load variables
design_crest_elevation= crest_elevation_5
#print('design value crest elevation=', design_crest_elevation.round(2), 'm')
design_bottom_elevation= bottom_elevation_95
#print('design value bottom elevation=', design_bottom_elevation.round(2), 'm')

In [ ]:
table = [['Variable', '5% exc. value', '95% exc. value', 'Design value'], ['crest elevation', crest_elevation_5.round(2), crest_elevation_95.round(2), design_crest_elevation.round(2)], 
         ['bottom elevation', bottom_elevation_5.round(2), bottom_elevation_95.round(2), design_bottom_elevation.round(2)]]
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒══════════════════╤═════════════════╤══════════════════╤════════════════╕
│ Variable         │   5% exc. value │   95% exc. value │   Design value │
╞══════════════════╪═════════════════╪══════════════════╪════════════════╡
│ crest elevation  │           13.84 │            14.16 │          13.84 │
├──────────────────┼─────────────────┼──────────────────┼────────────────┤
│ bottom elevation │          -34.93 │           -25.07 │         -25.07 │
╘══════════════════╧═════════════════╧══════════════════╧════════════════╛


<h2 style="color:#00BFFF;">1.5 The Limit State Function


</h2>
<hr style="height:1.5px;border:none;color:#333;" />

**<h3 style="color:#00BFFF;">Q5. Write the limit state function for overtopping.**
<hr style="height:1.5px;border:none;color:#333;" /> 

## Probabilistic calculation

First set up the marginal distributions (plot water level), then create the limit-state function and the multivariate probability distribution for use with OT.